In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/RunUNet

/content/drive/MyDrive/RunUNet


In [ ]:
ls

dataset.py  model.py  __pycache__/  saved_images/  utils.py


In [ ]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import numpy as np
from model import UNET
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders,
    check_accuracy,
    save_predictions_as_imgs,
)

# # Hyperparameters etc.
# LEARNING_RATE = 1e-4
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# BATCH_SIZE = 16
# NUM_EPOCHS = 3
# NUM_WORKERS = 2
# IMAGE_HEIGHT = 600
# IMAGE_WIDTH = 450
# PIN_MEMORY = True
# LOAD_MODEL = False

TRAIN_IMG_PATH = "/content/drive/MyDrive/UNet2/Data/trainimages.npy"
TRAIN_MASK_PATH = "/content/drive/MyDrive/UNet2/Data/trainmasks.npy"
VAL_IMG_PATH = "/content/drive/MyDrive/UNet2/Data/valimages.npy"
VAL_MASK_PATH = "/content/drive/MyDrive/UNet2/Data/valmasks.npy"

def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE, dtype=torch.float32)
        targets = targets.float().to(device=DEVICE) / 255.0

        # forward
        with torch.amp.autocast(device_type="cuda"):
          predictions = model(data)
          loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
print("Train images shape:", np.load(TRAIN_IMG_PATH).shape)
print("Train masks shape:", np.load(TRAIN_MASK_PATH).shape)

print("Unique values in masks:", np.unique(np.load(TRAIN_MASK_PATH)))
print("Any NaNs in images:", np.isnan(np.load(TRAIN_IMG_PATH)).sum())
print("Any NaNs in masks:", np.isnan(np.load(TRAIN_MASK_PATH)).sum())

Train images shape: (500, 450, 600, 3)
Train masks shape: (500, 450, 600, 1)
Unique values in masks: [  0 255]
Any NaNs in images: 0
Any NaNs in masks: 0


In [ ]:
# Hyperparameters etc.
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 4
NUM_WORKERS = 2
IMAGE_HEIGHT = 600
IMAGE_WIDTH = 450
PIN_MEMORY = True
LOAD_MODEL = False

In [ ]:
if __name__ == "__main__":
    train_transform = A.Compose([
        A.Resize(450, 600),  # Ensure correct dimensions
        A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0),
        ToTensorV2()
    ])

    val_transform = A.Compose([
        A.Resize(450, 600),  # Ensure correct dimensions
        A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0),  # Apply normalization to the image
        ToTensorV2()
    ])

    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_PATH, TRAIN_MASK_PATH, VAL_IMG_PATH, VAL_MASK_PATH,
        BATCH_SIZE, train_transform, val_transform, NUM_WORKERS, PIN_MEMORY
    )

    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)
        checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
        check_accuracy(val_loader, model, device=DEVICE)
        save_predictions_as_imgs(val_loader, model, folder="saved_images/", device=DEVICE)

<ipython-input-9-698c4113cd58>:23: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
100%|██████████| 32/32 [01:45<00:00,  3.29s/it, loss=0.462]


Got 91654901/135000000 with acc 67.89%
Dice score: 1.449852466583252


100%|██████████| 32/32 [02:04<00:00,  3.90s/it, loss=0.728]


Got 96730692/135000000 with acc 71.65%
Dice score: 1.1943962574005127


100%|██████████| 32/32 [02:04<00:00,  3.90s/it, loss=0.182]


Got 97986144/135000000 with acc 72.58%
Dice score: 1.2665032148361206


100%|██████████| 32/32 [02:04<00:00,  3.90s/it, loss=0.173]


Got 97523203/135000000 with acc 72.24%
Dice score: 1.3805286884307861


In [ ]:
save_checkpoint(checkpoint)

=> Saving checkpoint
